# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [3]:
#Definition of the tables.
import pandas as pd

data = {
    'table': ['customers', 'orders', 'products'],
    'definition': [
        'Contains customer details like name, email, and ID.',
        'Tracks customer orders including order date and product ID.',
        'Lists all products with price and stock availability.'
    ]
}

df = pd.DataFrame(data)
print(df)

       table                                         definition
0  customers  Contains customer details like name, email, an...
1     orders  Tracks customer orders including order date an...
2   products  Lists all products with price and stock availa...


In [4]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [5]:
print(text_tables)

customers: Contains customer details like name, email, and ID.
orders: Tracks customer orders including order date and product ID.
products: Lists all products with price and stock availability.


In [6]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [11]:
# Variant A – Straightforward
pqt_varA = prompt_question_tables.format(
    tables=text_tables,
    question="List the names and emails of customers who placed an order in the last 90 days."
)
print(return_OAI(pqt_varA))

# Variant B – Conversational
pqt_varB = prompt_question_tables.format(
    tables=text_tables,
    question="Which customers bought anything over $250, and what did they buy?"
)
print(return_OAI(pqt_varB))

# Variant C – Ultra-explicit (adds constraints)
pqt_varC = prompt_question_tables.format(
    tables=text_tables,
    question=("Return customer_id, first_name, last_name, order_date and product_name "
              "for orders where product_name LIKE '%Keyboard%' "
              "AND order_date BETWEEN '2024-01-01' AND CURRENT_DATE;")
)
print(return_OAI(pqt_varC))


{
    "tables": ["customers", "orders"]
}
{
    "tables": ["customers", "orders", "products"]
}
```json
{
    "tables": ["customers", "orders", "products"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

It's a balancing act


What I learned:

Prompt specificity reduces hallucinations.

The more structure (DDL, examples, constraints) you feed, the fewer phantom columns appear.

Few-shot examples help — giving one worked SQL pair upfront made GPT consistently output correct joins.

Natural-language clarity matters. Even a casual tone works if table names are present.

Temperature = 0 keeps it factual; anything above 0.4 increased creative but incorrect column names.

Always sanity-check: run the generated SQL or at least parse it against the schema before trusting it.

Take-away:

GPT is solid when you (a) include the schema in SQL DDL form, (b) give date/price constraints explicitly, and (c) keep questions single-intent. It drifts when context is thin.